In [2]:
from IPython.display import display, Math, Latex
from IPython.core.display import HTML 

In [3]:
# подгрузим модули
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

In [4]:
dataset = pd.read_csv('./data/ONLINEADS.csv', delimiter=';')

In [5]:
# посомтрим на датасет
dataset.head()

,Client ID,Age,Gender,Interest,VisitTime,City,Device,OS,VisitPage,AdsTool,VisitNumber,ViewedPage,Registration
0,1,0,1,0,14,1,0,1,0,0,0,0,1
1,2,0,2,0,8,0,0,1,1,2,1,2,1
2,3,2,1,0,21,1,0,1,0,3,0,2,1
3,4,0,1,0,23,1,0,0,1,3,1,0,1
4,5,2,0,0,12,0,0,1,0,0,1,4,1


In [6]:
dataset.fillna(method = 'ffill',axis = 1, inplace = True)

Видим, что датасет у нас представляет разреженную матрицу

In [7]:
dataset.head()

,Client ID,Age,Gender,Interest,VisitTime,City,Device,OS,VisitPage,AdsTool,VisitNumber,ViewedPage,Registration
0,1,0,1,0,14,1,0,1,0,0,0,0,1
1,2,0,2,0,8,0,0,1,1,2,1,2,1
2,3,2,1,0,21,1,0,1,0,3,0,2,1
3,4,0,1,0,23,1,0,0,1,3,1,0,1
4,5,2,0,0,12,0,0,1,0,0,1,4,1


In [8]:
#создаим из них матрицу
transactions = []
for i in range(0, 20000): 
    transactions.append([str(dataset.values[i,j]) for j in range(0, 13)])

In [9]:
pip install apriori

Note: you may need to restart the kernel to use updated packages.


In [12]:
#загружаем apriori
import apyori
from apyori import apriori

In [13]:
%%time
# и обучимся правилам. Обратите внимание, что пороговые значения мы вибираем сами в зависимости от того, /
# насколкьо "сильные" правила мы хотим получить
# min_support -- минимальный support для правил (dtype = float).
# min_confidence -- минимальное значение confidence для правил (dtype = float)
# min_lift -- минимальный lift (dtype = float)
# max_length -- максимальная длина itemset (вспоминаем про k-itemset)  (dtype = integer)

result = list(apyori.apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 4, min_length = 2))

Wall time: 11.8 s


Визуализируем выход

In [14]:
import shutil, os 

In [15]:
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

In [16]:
import json #преобразовывать будем в json, используя встроенные в модуль методы

In [17]:
output = []
for RelationRecord in result:
    o = StringIO()
    apyori.dump_as_json(RelationRecord, o)
    output.append(json.loads(o.getvalue()))
data_df = pd.DataFrame(output)
print(data_df)

Empty DataFrame
Columns: []
Index: []


In [18]:
# и взгялнем на итоги
pd.set_option('display.max_colwidth', None)

from IPython.display import display, HTML

display(HTML(data_df.to_html()))

ValueError: Value must have type '<class 'int'>'

Итого мы видим:

1. Пары items
2. items_base - первый элемент пары
3. items_add - второй (добавленный алгоритмом) элемент пары
4. confidence - значение confidence для пары
5. lift - значение lift для пары
6. support - начение support для пары. При желании, по нему можно отсортировать 


Результаты логичные: эскалоп и макароны, эскалоп и сливочно-грибной соус, курица и нежирная сметана, мягкий сыр и мед и т.д. - все это вполне логичные и, главное, вкусные сочетания:)

### ECLAT Algorithm

#### Реализация в Python

In [19]:
import numpy as np
"""
Класс инициируется 3мя параметрами:
- min_supp - минимальный support  который мы рассматриваем для ItemSet. Рассчитывается как % от количества транзакций
- max_items - максимальное количество елементов в нашем ItemSet
- min_items - минимальное количество элементов ItemSet
"""
class Eclat:
    #инициализация объекта класса
    def __init__(self, min_support = 0.01, max_items = 5, min_items = 2):
        self.min_support = min_support
        self.max_items = max_items
        self.min_items = min_items
        self.item_lst = list()
        self.item_len = 0
        self.item_dict = dict()
        self.final_dict = dict()
        self.data_size = 0
    
    #создание словаря из ненулевых объектов из всех транзакций (вертикальный датасет)
    def read_data(self, dataset):
        for index, row in dataset.iterrows():
            row_wo_na = row.dropna().unique()
            for item in row_wo_na:
                print(item)
                item = str(item).strip()
                if item in self.item_dict:
                    self.item_dict[item][0] += 1
                else:
                    self.item_dict.setdefault(item, []).append(1)
                self.item_dict[item].append(index)
        #задаем переменные экземпляра (instance variables)
        self.data_size = dataset.shape[0]
        self.item_lst = list(self.item_dict.keys())
        self.item_len = len(self.item_lst)
        self.min_support = self.min_support * self.data_size
        #print ("min_supp", self.min_support)
        
    #рекурсивный метод для поиска всех ItemSet по алгоритму Eclat
    #структура данных: {Item: [Supp number, tid1, tid2, tid3, ...]}
    def recur_eclat(self, item_name, tids_array, minsupp, num_items, k_start):
        if tids_array[0] >= minsupp and num_items <= self.max_items:
            for k in range(k_start+1, self.item_len):
                if self.item_dict[self.item_lst[k]][0] >= minsupp:
                    new_item = item_name + " | " + self.item_lst[k]
                    new_tids = np.intersect1d(tids_array[1:], self.item_dict[self.item_lst[k]][1:])
                    new_tids_size = new_tids.size
                    new_tids = np.insert(new_tids, 0, new_tids_size)
                    if new_tids_size >= minsupp:
                        if num_items >= self.min_items: self.final_dict.update({new_item: new_tids})
                        self.recur_eclat(new_item, new_tids, minsupp, num_items+1, k)
    
    #последовательный вызов функций определенных выше
    def fit(self, dataset):
        i = 0
        self.read_data(dataset)
        for w in self.item_lst:
            self.recur_eclat(w, self.item_dict[w], self.min_support, 2, i)
            i+=1
        return self
        
    #вывод в форме словаря {ItemSet: support(ItemSet)}
    def transform(self):
        return {k: "{0:.4f}%".format((v[0]+0.0)/self.data_size*100) for k, v in self.final_dict.items()}

Потестируем

In [20]:
#создадим экземпляр класса с нужными нам параметрами
model = Eclat(min_support = 0.01, max_items = 4, min_items = 3)

In [21]:
#обучим
model.fit(dataset)

1
0
14
2
0
8
1
3
2
1
0
21
4
0
1
23
3
5
2
0
12
1
4
6
2
0
11
1
3
7
2
1
18
0
3
8
2
1
0
9
2
1
0
14
3
10
1
2
0
18
11
0
2
15
1
12
3
2
0
13
1
13
2
0
10
5
1
14
0
1
8
3
4
15
2
0
17
1
16
1
2
0
21
17
0
1
13
4
18
2
0
5
1
19
0
1
10
20
2
0
17
1
21
1
2
18
0
5
22
1
4
21
0
2
23
0
1
8
5
24
1
2
22
0
4
3
25
0
1
3
7
26
2
0
21
1
27
2
0
18
1
3
28
0
1
12
3
4
29
2
0
15
1
30
2
1
22
0
6
31
1
0
5
2
32
2
0
12
1
5
3
33
0
1
9
3
34
1
0
10
2
35
0
2
3
18
1
4
36
0
1
14
2
37
0
2
4
9
1
38
0
1
14
5
39
3
1
0
12
5
4
40
0
19
1
2
41
2
1
0
8
3
42
2
0
10
1
43
0
2
18
1
5
6
44
2
1
0
12
3
45
3
1
0
17
4
46
0
10
1
2
4
47
0
2
5
1
3
48
0
1
4
16
3
49
2
1
0
12
50
0
8
1
2
3
51
3
2
12
0
1
52
0
2
19
1
5
4
53
0
2
1
11
4
54
0
1
6
55
0
2
18
1
4
56
2
0
12
1
3
57
0
1
21
3
58
1
2
16
0
59
2
1
0
16
60
0
2
20
1
3
61
1
2
16
0
3
62
1
0
12
3
63
0
1
19
4
64
1
0
18
5
65
2
1
3
10
0
66
2
0
21
1
67
2
1
0
14
68
2
1
0
17
69
0
2
14
1
70
0
1
19
3
71
0
1
2
8
72
2
0
7
1
4
73
0
2
3
11
1
74
0
16
1
75
0
2
21
3
1
76
1
2
0
21
5
77
1
2
0
18
78
3
0
10
1
2
79
2
1
0
16
80

In [22]:
#и визуализируем результаты
model.transform()

{'1 | 0 | 14': '5.5700%',
 '1 | 0 | 14 | 2': '4.2250%',
 '1 | 0 | 14 | 3': '2.5950%',
 '1 | 0 | 14 | 4': '1.7550%',
 '1 | 0 | 14 | 5': '1.1900%',
 '1 | 0 | 2': '78.6050%',
 '1 | 0 | 2 | 8': '8.2450%',
 '1 | 0 | 2 | 3': '38.4500%',
 '1 | 0 | 2 | 21': '3.2000%',
 '1 | 0 | 2 | 4': '29.9200%',
 '1 | 0 | 2 | 23': '2.3300%',
 '1 | 0 | 2 | 5': '21.6950%',
 '1 | 0 | 2 | 12': '4.1700%',
 '1 | 0 | 2 | 6': '14.2450%',
 '1 | 0 | 2 | 11': '4.1800%',
 '1 | 0 | 2 | 7': '7.8050%',
 '1 | 0 | 2 | 18': '3.8350%',
 '1 | 0 | 2 | 9': '7.9200%',
 '1 | 0 | 2 | 10': '4.2450%',
 '1 | 0 | 2 | 15': '4.2700%',
 '1 | 0 | 2 | 13': '4.3550%',
 '1 | 0 | 2 | 17': '3.5050%',
 '1 | 0 | 2 | 16': '4.1300%',
 '1 | 0 | 2 | 19': '3.7350%',
 '1 | 0 | 2 | 20': '3.4750%',
 '1 | 0 | 2 | 22': '2.9800%',
 '1 | 0 | 8': '9.8500%',
 '1 | 0 | 8 | 3': '4.8250%',
 '1 | 0 | 8 | 4': '3.8450%',
 '1 | 0 | 8 | 5': '2.6900%',
 '1 | 0 | 8 | 6': '1.6200%',
 '1 | 0 | 3': '49.6700%',
 '1 | 0 | 3 | 21': '2.0100%',
 '1 | 0 | 3 | 4': '16.6400%',
 '1 

Как видно, реализовать алгоритм своими силами довольно просто, хотя с эффективностью стоит поработать:)

### FP-Growth Algorithm

#### Реализация в Python

In [23]:
pip install pyfpgrowth

Note: you may need to restart the kernel to use updated packages.


In [24]:
import pyfpgrowth

In [25]:
dataset2 = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

In [28]:
#Сгенериуем паттерны
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset2).transform(dataset2)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


In [29]:
#Сгенериуем правила
from mlxtend.frequent_patterns import fpgrowth

fpgrowth(df, min_support=0.01)

,support,itemsets
0,1.0,(5)
1,0.8,(3)
2,0.6,(10)
3,0.6,(8)
4,0.6,(6)
5,0.4,(7)
6,0.2,(2)
7,0.2,(0)
8,0.4,(1)
9,0.2,(9)


In [30]:
fpgrowth(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,1.0,(Kidney Beans)
1,0.8,(Eggs)
2,0.6,(Yogurt)
3,0.6,(Onion)
4,0.6,(Milk)
5,0.8,"(Eggs, Kidney Beans)"
6,0.6,"(Yogurt, Kidney Beans)"
7,0.6,"(Onion, Eggs)"
8,0.6,"(Onion, Kidney Beans)"
9,0.6,"(Onion, Eggs, Kidney Beans)"
